In [ ]:
import sys
!pip install --prefix {sys.prefix} ismn

In [1]:
import xarray as xr
import pandas as pd
from glob import glob
import geojson

## ismn

In [2]:
from ismn.interface import ISMN_Interface
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [3]:
# Path to data downloaded from ismn network in header+value composite
file_name_in = 'Data_separate_files_header_20150101_20211216_9207_57tZ_20211216.zip'
path_ismn_zip = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/ismn_archieve/'
file_name_out = 'coords.geojson'

In [4]:
# Either a .zip file or one folder that contains all networks, here we read from .zip
ismn_data = ISMN_Interface(path_ismn_zip + file_name_in, parallel=True)

Found existing ismn metadata in C:\Users\USER\Desktop\Master_Irrigation\03_GIS\ground_trouth\ismn_archieve\python_metadata\Data_separate_files_header_20150101_20211216_9207_57tZ_20211216.csv.


In [6]:
ismn_data.networks

OrderedDict([('BIEBRZA_S-1',
              Stations in 'BIEBRZA_S-1': ['grassland_meteo_0', 'grassland_soil_1', 'grassland_soil_2', 'grassland_soil_3', 'grassland_soil_4', 'grassland_soil_5', 'grassland_soil_6', 'grassland_soil_7', 'grassland_soil_8', 'grassland_soil_9', 'marshland_meteo_10', 'marshland_meteo_20', 'marshland_soil_11', 'marshland_soil_12', 'marshland_soil_13', 'marshland_soil_14', 'marshland_soil_15', 'marshland_soil_16', 'marshland_soil_17', 'marshland_soil_18', 'marshland_soil_19', 'marshland_soil_21', 'marshland_soil_22', 'marshland_soil_23', 'marshland_soil_24', 'marshland_soil_25', 'marshland_soil_26', 'marshland_soil_27', 'marshland_soil_28', 'marshland_soil_29']),
             ('FMI',
              Stations in 'FMI': ['SAA111', 'SAA112', 'SAA120', 'SOD011', 'SOD012', 'SOD013', 'SOD021', 'SOD022', 'SOD023', 'SOD031', 'SOD032', 'SOD033', 'SOD071', 'SOD072', 'SOD073', 'SOD081', 'SOD082', 'SOD083', 'SOD091', 'SOD092', 'SOD093', 'SOD101', 'SOD102', 'SOD103', 'SOD130',

In [7]:
# Print this line if you want to know which landcover and climate classes are available
ismn_data.print_climate_dict(), ismn_data.print_landcover_dict()

KOEPPEN GEIGER Climate Classification
-------------------------------------
Af  : Tropical Rainforest
Am  : Tropical Monsoon
As  : Tropical Savanna Dry
Aw  : Tropical Savanna Wet
BWk : Arid Desert Cold
BWh : Arid Desert Hot
BWn : Arid Desert With Frequent Fog
BSk : Arid Steppe Cold
BSh : Arid Steppe Hot
BSn : Arid Steppe With Frequent Fog
Csa : Temperate Dry Hot Summer
Csb : Temperate Dry Warm Summer
Csc : Temperate Dry Cold Summer
Cwa : Temperate Dry Winter, Hot Summer
Cwb : Temperate Dry Winter, Warm Summer
Cwc : Temperate Dry Winter, Cold Summer
Cfa : Temperate Without Dry Season, Hot Summer
Cfb : Temperate Without Dry Season, Warm Summer
Cfc : Temperate Without Dry Season, Cold Summer
Dsa : Cold Dry Summer, Hot Summer
Dsb : Cold Dry Summer, Warm Summer
Dsc : Cold Dry Summer, Cold Summer
Dsd : Cold Dry Summer, Very Cold Winter
Dwa : Cold Dry Winter, Hot Summer
Dwb : Cold Dry Winter, Warm Summer
Dwc : Cold Dry Winter, Cold Summer
Dwd : Cold Dry Winter, Very Cold Winter
Dfa : Cold Dry

(None, None)

In [35]:
# Search for ids with specific condtions
ids = ismn_data.get_dataset_ids(variable='soil_moisture',
                                max_depth=0.05,
                                filter_meta_dict={'lc_2005': [10,11,20,130,200, 201, 202, 30], 'climate_KG': ['Csa', 'Csb', 'Dwa', 'Dwb']})
len(ids)

5

In [36]:
ids

[1341, 1348, 1825, 1832, 1862]

In [37]:
# Read Ids
ts = [ismn_data.read(x ,return_meta=True) for x in ids]
ts_data = [x[0] for x in ts]
ts_meta = pd.DataFrame([x[1] for x in ts]).set_index('station')

In [47]:
ts[2][0]

,soil_moisture,soil_moisture_flag,soil_moisture_orig_flag
date_time,,,
2018-06-11 00:00:00,0.062,G,M
2018-06-11 01:00:00,0.062,G,M
2018-06-11 02:00:00,0.062,G,M
2018-06-11 03:00:00,0.061,G,M
2018-06-11 04:00:00,0.062,G,M
...,...,...,...
2019-12-31 20:00:00,0.131,G,M
2019-12-31 21:00:00,0.131,G,M
2019-12-31 22:00:00,0.131,G,M


In [38]:
#save ts data with id and lat_lon bezeichnung
for (data, meta), ismn_id in zip(ts, ids):
    data.to_json(path_ismn_zip + f'{ismn_id}_{meta.longitude.values[0]}_{meta.latitude.values[0]}_{meta.network.values[0]}.json')

In [ ]:
import geopandas as gpd

In [ ]:
import geojson

# Create Geojson MultiPoint FeatureCollection 
geodf = gpd.GeoDataFrame(data=None, geometry=gpd.points_from_xy(x=ts_meta['longitude'].unstack().values, y=ts_meta['latitude'].unstack().values))
fc = geodf.to_json()
# Save geojson to disk
with open(path_ismn_zip + file_name_out, 'w') as f:
    geojson.dump(fc, f)

In [ ]:
# Quality Controle
#Qflag
#NDVI < 0.7


In [ ]:
#Plot Specific ids metdata and time series
ts, meta = ismn_data.read(ids[3], return_meta=True)
ax = ts.plot(figsize=(12,4), title=f'Time series for ID {ids[3]}')
ax.set_xlabel("Time [year]")
ax.set_ylabel("Soil Moisture")
meta,ts

In [ ]:
import cartopy.crs as ccrs
#plot available station on a map
fig, axs = plt.subplots(1, 2, figsize=(16,10), subplot_kw={'projection': ccrs.Robinson()})
ismn_data.plot_station_locations('soil_moisture', min_depth=0., max_depth=0.2, ax=axs[0])
plt.show()

In [ ]:
gt_path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/'
#irrigation # rainfall 
path_brandenburg = gt_path + 'irrigation+_archive/In situ data/German/Irrigation_Test_Site_Brandenburg.nc'
path_niedersachsen = gt_path + 'irrigation+_archive\Input netcdf\TEST_SITE_GERMANY_NIEDERSACHSEN.nc'
path_budrio = gt_path + 'irrigation+_archive/In situ data/Italy/Irrigation_Test_Site_Budrio.nc'

# Tereno
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/TERENO_archieve'
files_BF = glob(path + '/*BF*')

In [ ]:
#Tereno
test = pd.read_csv('C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/TERENO_archieve/altPlestlin_BF1_v1.2_20121029_20210630/altPlestlin_BF1_v1.2_20121029_20210630.csv')
test.columns

In [ ]:
brandenburg = xr.load_dataset(path_niedersachsen)
#field_1 = brandenburg[['Longitude_1', 'Latitude_1', 'Time-days_1']]
brandenburg

In [ ]:
field_1